In [1]:
import laspy
import numpy as np
from scipy.interpolate import griddata
from pyproj import Transformer, CRS
from tqdm import tqdm
import pandas as pd
import sys
import pygmt
from sklearn.cluster import DBSCAN
import utm

In [2]:
# Define conversion functions

def utm_to_latlon(x, y):
    # Convert lat/lon to UTM coordinates
    lat, lon = utm.to_latlon(x, y, 32, 'U')

    return lat, lon

def latlon_to_utm(lat, lon):
    # Convert lat/lon to UTM coordinates
    utm_x, utm_y, _, _ = utm.from_latlon(lat, lon)

    return utm_x, utm_y

In [17]:
# Example usage
# laz_file_path = "../lasersurface/lidar_data/3dm_32_356_5645_1_nw.laz" # Dom

# laz_file_path = "../lasersurface/lidar_data/3dm_32_358_5643_1_nw.laz" # TÜV building in Poll

laz_file_path = '/Volumes/SSD2/Split_NRW/zone_1.3/3dm_32_336_5651_1_nw.laz'


In [18]:
with laspy.open(laz_file_path) as file:
    las = file.read()

In [19]:
SSFACTOR = 1 # Subsampling factor for points cloud

lastReturnNichtBoden = 20
brueckenpunkte = 17
unclassified = 1

class_ok = [brueckenpunkte, lastReturnNichtBoden, unclassified]

class_val = las.classification[::SSFACTOR]

mask = (np.isin(class_val, class_ok))

points = np.vstack((las.x[::SSFACTOR][mask], las.y[::SSFACTOR][mask], las.z[::SSFACTOR][mask])).transpose()

ground_points = las.points[las.classification == 2]

gnd_points = np.vstack((ground_points.x, ground_points.y, ground_points.z)).transpose()


In [20]:
GCG2016_geoid_file = './GCG2016_data/GCG2016_we.tif'
backup_DEM_file = './DEM_data/urn_eop_DLR_CDEM10_Copernicus_DSM_04_N50_00_E006_00_V8239-2020_1__DEM1__coverage_20231204210410.tif'
#DEM_file = '/Volumes/SSD2/Split_NRW/dgm1_32_358_5643_1_nw.tif'
DEM_file = '/Volumes/SSD2/Split_NRW/zone_1.3_DEM/dgm1_32_336_5651_1_nw.tif'
egm96_geoid_file = './EGM96_data/egm96_15.gtx'
egg08_geoid_file = './EGG08_data/egm08_25.gtx'

In [21]:
# Create temporary reprojected DEM file (EPSG:28532 to EPSG:4326)
# source: https://rasterio.readthedocs.io/en/latest/topics/reproject.html
import numpy as np
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

dst_crs = 'EPSG:4326'

with rasterio.open(DEM_file) as src:
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    with rasterio.open('./temp_DEM_file.tif', 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest)

In [22]:
temp_DEM_file = './temp_DEM_file.tif'

transformer = Transformer.from_pipeline(
    f"+proj=pipeline "
    f"+step +inv +proj=utm +zone=32 +ellps=WGS84 "  # Convert from UTM Zone 32N to geographic coordinates
    f"+step +proj=vgridshift +grids=@{temp_DEM_file},{backup_DEM_file} +multiplier = -1 "  # Vertical grid shift to remove the DEM elevation
)

# Tests

In [109]:
transformed_z = np.array([transformer.transform(xi, yi, zi)[2] for xi, yi, zi in tqdm(zip(las.x, las.y, las.z), total=9471722)])


100%|██████████| 9471722/9471722 [00:37<00:00, 252318.18it/s]


In [23]:
transformer.transform(356000.10000000003, 5645018.83,200)

(inf, inf, inf)

In [110]:
transformed_z.mean()

2.248118481420427

## test to transform only ground points and check average height

In [111]:
transformed_ground_z = np.array([transformer.transform(xi, yi, zi)[2] for xi, yi, zi in tqdm(zip(ground_points.x, ground_points.y, ground_points.z), total=len(ground_points.x))])

100%|██████████| 3365666/3365666 [00:13<00:00, 253146.69it/s]


In [117]:
transformed_ground_z.mean()

-4.416774161873778

## Check DEM file values with rasterio

In [45]:
import rasterio
from rasterio.transform import from_origin

def get_dem_elevation(dem_file, lat, lon):
    """
    Get the elevation from a DEM file at a given latitude and longitude.

    Parameters:
    dem_file (str): Path to the DEM file.
    lat (float): Latitude of the point.
    lon (float): Longitude of the point.

    Returns:
    float: Elevation at the given latitude and longitude.
    """
    with rasterio.open(dem_file) as dataset:
        # Convert the latitude and longitude to row and column
        row, col = dataset.index(lon, lat)

        print(row)
        print(col)

        row -= 5645000
        col *= -1
        col -= 356000

        # Read the elevation at the given row and column
        elevation = dataset.read(1)[row, col]
        return elevation

In [27]:
with rasterio.open(DEM_file) as dataset:
    print(dataset.crs)

    bounds = dataset.bounds
    print("Bounds of the dataset:", bounds)

EPSG:25832
Bounds of the dataset: BoundingBox(left=356000.0, bottom=5645000.0, right=357000.0, top=5646000.0)


In [48]:
latitude, longitude = 50.941276,6.957772
elevation = get_dem_elevation(DEM_file, latitude, longitude)
print(f"Elevation at ({latitude}, {longitude}): {elevation} meters")

5645949
-355994
Elevation at (50.941276, 6.957772): 39.20750045776367 meters


# Clustering

In [23]:
#transformed_points = np.array([transformer.transform(xi, yi, zi) for xi, yi, zi in tqdm(zip(las.x, las.y, las.z), total=len(las.x))])

import os

total = len(points[:,0])
transformed_points = np.array([transformer.transform(xi, yi, zi) for xi, yi, zi in tqdm(zip(points[:,0], points[:,1], points[:,2]), total = total)])

#os.remove(temp_DEM_file)

100%|██████████| 5211202/5211202 [00:16<00:00, 308825.46it/s]


In [95]:
points[:,0][1234], points[:,1][1234]

(365062.43, 5619009.24)

In [24]:
# Create our dataframe

df = pd.DataFrame(
    data={
        "x": points[:,0], #np.array(las.x), # We need UTM coordinates
        "y": points[:,1], #np.array(las.y), # 
        "z": points[:,2],
        "h": transformed_points[:,2]
    }
)

size_df = sys.getsizeof(df)
print(f"Size of the DataFrame: {np.ceil(size_df / (1024*1024))} MB")

Size of the DataFrame: 160.0 MB


In [25]:
inf_rows = df.isin([np.inf, -np.inf]).any(axis=1)
inf_rows_df = df[inf_rows]
df = df[~inf_rows]
print(f'Removed {len(inf_rows_df)} rows to the dataframe')


Removed 0 rows to the dataframe


In [26]:
region = pygmt.info(data=df[["x", "y"]], spacing=1)  # West, East, South, North

print(f"Data points covers region: {region}")

Data points covers region: [ 336000.  337000. 5651000. 5652000.]


In [27]:
x_min, x_max, y_min, y_max = list(region)

In [28]:
# Filtering the DataFrame
condition = (abs(df['x'] - x_min) < 1.5) | \
            (abs(df['x'] - x_max) < 1.5) | \
            (abs(df['y'] - y_min) < 1.5) | \
            (abs(df['y'] - y_max) < 1.5)

df_filtered = df[~condition]

# Number of rows removed
print(f'Removed another {len(df) - len(df_filtered)} points on the edge')


Removed another 28243 points on the edge


In [29]:
df_trimmed = pygmt.blockmedian(
    data=df_filtered[["x", "y", "h"]],
    T=0.9999,  # 99.99th quantile, i.e. the highest point
    spacing="1+e", # 1+e for 1 m # 0.1 increases the size of df but more accurate?
    region=region,
)

size_df_trimmed = sys.getsizeof(df_trimmed)
print(f"Size of the DataFrame: {np.ceil(size_df_trimmed / (1024*1024))} MB")

Size of the DataFrame: 14.0 MB


In [36]:
# Identify points that are above 120m above geoid (for Cologne this means about 70 m above ground).
high_points = df_trimmed[df_trimmed['h'] > 90] # Default = 60

# Assuming that points within 100m of each other belong to the same obstacle
clustering = DBSCAN(eps=40, min_samples=2).fit(high_points[['x', 'y', 'h']]) # TODO: no error if no cluster found # Default = 50

# Add the cluster labels to the high_points DataFrame
high_points['cluster'] = clustering.labels_

# Filter out noise points (DBSCAN labels noise as -1)
obstacles = high_points[high_points['cluster'] != -1]

# Find the highest point in each obstacle cluster
highest_points = obstacles.loc[obstacles.groupby('cluster')['h'].idxmax()]

# The resulting DataFrame 'highest_points' contains the coordinates of the highest point of each obstacle
highest_points.reset_index(drop=True, inplace=True)

# Apply the conversion function to the DataFrame to create new columns 'lat' and 'lon'
highest_points['lat'], highest_points['lon'] = zip(*highest_points.apply(lambda row: utm_to_latlon(row['x'], row['y']), axis=1))

# Display the resulting DataFrame
pd.set_option('display.max_rows', 200)
highest_points

/var/folders/v_/0jkxx12x7r9_3szb050fks9m0000gn/T/ipykernel_64155/13985715.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,x,y,h,cluster,lat,lon
0,336559.08,5651952.44,154.513884,0,50.995911,6.670845
1,336396.48,5651892.51,127.371766,1,50.995327,6.668557
2,336485.25,5651856.49,131.295467,2,50.995028,6.669837
3,336223.13,5651813.35,108.983335,3,50.994566,6.666124
4,336736.35,5651689.49,175.968173,4,50.993599,6.673487
5,336414.41,5651732.44,103.760000,5,50.993894,6.668884
6,336297.40,5651723.48,117.257434,6,50.993780,6.667222
7,336471.45,5651686.50,164.198531,7,50.993497,6.669717
8,336413.26,5651658.37,127.823727,8,50.993228,6.668901
9,336366.48,5651628.50,125.292718,9,50.992946,6.668249


In [62]:
df_filtered.h.median()

6.602780492487682

In [127]:
transformed_points[:,2].min()

-222.51415827978076

## Visualise obstacles

In [37]:
import plotly.graph_objects as go

scattermapbox_objects = []

scattermapbox_objects.append(go.Scattermapbox(
    mode="markers",
    lon=highest_points['lon'], 
    lat=highest_points['lat'],
    marker={'size': 20, 'color': "red", 'opacity': 0.5,},
    text='Cluster: '+ highest_points['cluster'].apply(lambda x: str(x)) + '<br>' + 'Calculated height (m): ' + highest_points['h'].apply(lambda x: str(round(x, 1))),
    hoverinfo='text'  # Only display the text on hover
))
'''
scattermapbox_objects.append(go.Scattermapbox(
    mode="markers",
    lon=AIP_df['geoLong'], 
    lat=AIP_df['geoLat'],
    marker={'size': 10, 
            'symbol': 'circle',
            'color': "blue",
            'opacity': 0.7,
           },
    text='Name: ' + AIP_df['txtName']+'<br>'+'Published height (m): ' + AIP_df['valHgt (m)'].apply(lambda x: str(round(x, 1))),
    hoverinfo='text'  # Only display the text on hover
))
'''
'''scattermapbox_objects.append(go.Scattermapbox(
        name = 'Data limits',
        mode="lines",
        line=dict(color="black", width=1),
        lat=np.array([LAT_MAX, LAT_MAX, LAT_MIN, LAT_MIN, LAT_MAX]),
        lon=np.array([LON_MIN, LON_MAX, LON_MAX, LON_MIN, LON_MIN]),
        hoverinfo='name',
        hoverlabel_namelength=-1   # https://stackoverflow.com/questions/36207887/plot-ly-hover-box-size-attribute
    ))
'''
# Create a scatter plot of the highest points using Plotly with OpenStreetMap background
fig = go.Figure(data=scattermapbox_objects)

# Set the layout for the map
fig.update_layout(
    mapbox={
        'style': "open-street-map",
        'center': {'lon': np.mean(highest_points['lon']), 'lat': np.mean(highest_points['lat'])},
        'zoom': 12
    },
    showlegend=False
)

# Adjust the margins and set the height
fig.update_layout(height=800, margin={"r":10,"t":10,"l":10,"b":10})

# Show the figure
fig.show()

## Check the transformation steps to understand their effect

In [38]:
transformer1 = Transformer.from_pipeline(
    f"+proj=pipeline "
    f"+step +inv +proj=utm +zone=32 +ellps=WGS84 "  # Convert from UTM Zone 32N to geographic coordinates
    f"+step +proj=vgridshift +grids={GCG2016_geoid_file} +multiplier = 1 "  # Vertical grid shift to add the DHHN16 elevation -> WGS84
)

transformer2 = Transformer.from_pipeline(
    f"+proj=pipeline "
    f"+step +inv +proj=utm +zone=32 +ellps=WGS84 "  # Convert from UTM Zone 32N to geographic coordinates
    f"+step +proj=vgridshift +grids={temp_DEM_file} +multiplier = -1 "  # Vertical grid shift to remove the DEM elevation
)

transformer3 = Transformer.from_pipeline(
    f"+proj=pipeline "
    f"+step +inv +proj=utm +zone=32 +ellps=WGS84 "  # Convert from UTM Zone 32N to geographic coordinates
    f"+step +proj=vgridshift +grids={egg08_geoid_file} +multiplier = -1 " # Vertical grid shift to remove the EGM2008 geoid height
)

transformer4 = Transformer.from_pipeline(
    f"+proj=pipeline "
    f"+step +inv +proj=utm +zone=32 +ellps=WGS84 "  # Convert from UTM Zone 32N to geographic coordinates
    f"+step +proj=vgridshift +grids={backup_DEM_file} +multiplier = -1 "  # Vertical grid shift to remove the DEM elevation
)

In [39]:
x, y = 336413.26, 5651658.37

print(
    f"Total shift: {transformer.transform(x, y,0)[2]}\n"
    f"Step 1: {transformer1.transform(x, y,0)[2]}\n"
    f"Step 2: {transformer2.transform(x, y,0)[2]}\n"
    f"Step 3: {transformer3.transform(x, y,0)[2]}\n"
    f"Step 4: {transformer4.transform(x, y,0)[2]}"
)

Total shift: -82.41323781315228
Step 1: 45.973466354370714
Step 2: -82.41323781315228
Step 3: -46.20117970245901
Step 4: -101.32412877564747


In [53]:
with rasterio.open(DEM_file) as dem:
    # Transformer to convert UTM to geographic coordinates (WGS 84)
    transformer = Transformer.from_crs("epsg:25832", "epsg:4326", always_xy=True)

    # UTM coordinates
    utm_x, utm_y = 356595.8,5645474.8

    # Convert UTM coordinates to row and column indices in the DEM
    row, col = dem.index(utm_x, utm_y)

    # Read the elevation value at the specified indices
    elevation = dem.read(1)[row, col]

    print("Elevation at the coordinate:", elevation)

525 595
Elevation at the coordinate: 55.56


# new attempt with original DEM file

In [139]:
def find_elev(x, y, dem_file):

    with rasterio.open(dem_file) as dem:
    # Transformer to convert UTM to geographic coordinates (WGS 84)

        # Convert UTM coordinates to row and column indices in the DEM
        row, col = dem.index(x, y)

        if row==1000: row=999
        if col==1000: col=999

        #print(x, y, row, col)

        # Read the elevation value at the specified indices
        elevation = dem.read(1)[row, col]

        return elevation

In [140]:
z_array = np.array([find_elev(x, y, DEM_file) for x, y in tqdm(zip(points[:,0], points[:,1]))])

# takes forever

624it [00:21, 208.55it/s]

KeyboardInterrupt: 